In [1]:
import cv2
import os
import numpy as np
from utils import get_video_path, get_images_dir_path, get_every_video_frame, get_image_prefix, timing_decorator

In [2]:
BLACK_PERCENTAGE = 30 # разрешенный процент черного цвета на фото
TARGET_IMAGE_FRAME = (96, 54) # целевой размер изображения

In [3]:
video_path = get_video_path()
images_path = get_images_dir_path()
every_video_frame = get_every_video_frame()
image_prefix = get_image_prefix()

In [4]:
# расчёт процента черного цвета на фото
def calculate_black_percentage(image, binary):
    black_pixels = np.sum(binary == 0)
    total_pixels = image.shape[0] * image.shape[1]

    black_percentage = (black_pixels / total_pixels) * 100

    return black_percentage

In [5]:
# обработка кадра
def process_frame(frame, target_size=TARGET_IMAGE_FRAME):
    resized_frame = cv2.resize(frame, target_size, interpolation=cv2.INTER_LINEAR)

    gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    _, binary = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)

    black_percentage = calculate_black_percentage(resized_frame, binary)

    if black_percentage > BLACK_PERCENTAGE:
        return None, True

    return binary, False

In [6]:
def create_image_path(output_folder, image_prefix, saved_frame_count):
    frame_filename = os.path.join(output_folder, f'{image_prefix}{saved_frame_count:05d}.jpg')
    return frame_filename

In [7]:
@timing_decorator
def save_frames(video_path, output_folder, image_prefix, every_video_frame):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Ошибка открытия видео {video_path}")
        return

    frame_count = 0
    saved_frame_count = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        if frame_count % every_video_frame == 0:
            frame, is_error = process_frame(frame)

            if is_error:
                continue

            frame_filename = create_image_path(output_folder, image_prefix, saved_frame_count)
            cv2.imwrite(frame_filename, frame)
            saved_frame_count += 1

        frame_count += 1

    cap.release()
    print(f"Сохранено {saved_frame_count} кадров в папку {output_folder}")

In [8]:
print(f"Началась обработка видео")
save_frames(video_path=video_path, output_folder=images_path, image_prefix=image_prefix,
                every_video_frame=every_video_frame)
print(f"Обработка видео завершена")

Началась обработка видео
Сохранено 47933 кадров в папку /Users/rrminyukov/PycharmProjects/ml-basics/dance-moves/images
Время выполнения save_frames: 46.55 секунд
Обработка видео завершена
